In [8]:
import numpy
import os

DIR = os.path.expanduser("~")
animal = 'MD589'

volume_path = os.path.join(DIR, f'{animal}_annotations.npy')
with open(volume_path, 'rb') as file:
    volume = numpy.load(file)

In [9]:
from cloudvolume import CloudVolume

precompute_path = os.path.join(DIR, f'{animal}_annotations')
cloudpath = f'file://{precompute_path}'
info = CloudVolume.create_new_info(
    num_channels = 1,
    layer_type = 'segmentation',
    data_type = str(volume.dtype), # Channel images might be 'uint8'
    encoding = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = [14464, 14464, 20000], # Voxel scaling, units are in nanometers
    voxel_offset = [0, 0, 0], # x,y,z offset in voxels from the origin
    chunk_size = [128, 128, 16], # units are voxels
    volume_size = volume.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(cloudpath, mip=0, info=info, compress=False)
vol.commit_info()
vol[:, :, :] = volume[:, :, :]

Uploading: 100%|██████████| 2464/2464 [00:13<00:00, 182.79it/s]


In [10]:
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc

tq = LocalTaskQueue(parallel=4)
tasks = tc.create_downsampling_tasks(cloudpath)
tq.insert(tasks)
tasks = tc.create_meshing_tasks(cloudpath, mip=0, compress=False)
tq.insert(tasks)
tasks = tc.create_mesh_manifest_tasks(cloudpath)
tq.insert(tasks)
tq.execute(progress=True)

Tasks:   0%|          | 0/1039 [00:00<?, ?it/s]

Volume Bounds:  Bbox([0, 0, 0],[1365, 1012, 447], dtype=int32)
Selected ROI:   Bbox([0, 0, 0],[1365, 1012, 447], dtype=int32)


Tasks: 100%|██████████| 1039/1039 [00:38<00:00, 26.86it/s]
